Aynur Rahimova

##### **Evaluating the Impact of Semiconductor Adoption and Technological Change on Occupational Shifts and Workforce Displacement Across Major Industries**

**In this study, a supervised regression model will be implemented**

**Used datasets**:

* **UN Comtrade database → Semiconductor adoption**:
https://comtradeplus.un.org/TradeFlow 

* **ILOSTAT Data Explorer – Employment by Occupation**:
https://rshiny.ilo.org/dataexplorer9/

* **Research and development expenditure(R&D) (% of GDP) indicator**:
https://data.worldbank.org/indicator/GB.XPD.RSDV.GD.ZS?end=2023&start=2000

##### **Notebook "Preprocessing"** (part 1)

**Import**

In [ ]:
# Import necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import category_encoders as ce
import pickle
from sklearn import set_config
from sklearn.compose import ColumnTransformer

**Preprocessing Semiconductor Adoption dataset before merging**

**Loading datasets**

Loading the dataset: UN Comtrade database → semiconductor adoption. 

It was not possible to download the complete dataset in 1 time, so i had to divide it into 3 datasets:

TradeData_2000_2011_HS8541_8542.csv

TradeData_2012_2023_HS8541_8542.csv

TradeData_2024_HS8541_8542.csv

In [ ]:
path_trade1 = "../Data/Row/TradeData_2000_2011_HS8541_8542.csv"
path_trade2 = "../Data/Row/TradeData_2012_2023_HS8541_8542.csv"
path_trade3 = "../Data/Row/TradeData_2024_HS8541_8542.csv"
df_trade1 = pd.read_csv(path_trade1, encoding='latin1')
df_trade2 = pd.read_csv(path_trade2, encoding='latin1')
df_trade3 = pd.read_csv(path_trade3, encoding='latin1')

Combining 3 divided datasets in 1 complete dataset

In [6]:
combined_df_base = pd.concat([df_trade1, df_trade2, df_trade3], ignore_index=True)

In [7]:
combined_df_semicond = combined_df_base.copy() # to avoid modifying the original dataframe

Before merging 3 datasets - Semiconductor adoption, Employment by occupation and Research and development expenditure(R&D) (% of GDP) indicator - I will clean and preprocess each of them separately to produce a high-quality, consistent and reliable resulting combined dataset. I will apply a detailed EDA to the final dataset.

**Cleaning and preprocessing of Semiconductor adoption dataset**

In [ ]:
# to check the first few rows of the dataframe

combined_df_semicond.head()       # The names of some columns do not correspond to the values they contain.

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate
0,A,20000101,2000,52,2000,8,ALB,Albania,M,Import,...,True,NaN,False,3161.0,NaN,3161.0,4,False,False,NaN
1,A,20000101,2000,52,2000,8,ALB,Albania,X,Export,...,True,NaN,False,NaN,1086.0,1086.0,4,False,False,NaN
2,A,20000101,2000,52,2000,12,DZA,Algeria,M,Import,...,False,NaN,False,3456956.0,NaN,3456956.0,0,False,False,NaN
3,A,20000101,2000,52,2000,12,DZA,Algeria,X,Export,...,False,NaN,False,NaN,1018.0,1018.0,0,False,False,NaN
4,A,20000101,2000,52,2000,20,AND,Andorra,M,Import,...,False,NaN,False,12786.0,NaN,12786.0,0,False,False,NaN


In [9]:
combined_df_semicond.info() # to get information about the DataFrame(number of rows, columns, data types)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14827 entries, 0 to 14826
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   typeCode                  14827 non-null  object 
 1   freqCode                  14827 non-null  int64  
 2   refPeriodId               14827 non-null  int64  
 3   refYear                   14827 non-null  int64  
 4   refMonth                  14827 non-null  int64  
 5   period                    14827 non-null  int64  
 6   reporterCode              14827 non-null  object 
 7   reporterISO               14827 non-null  object 
 8   reporterDesc              14827 non-null  object 
 9   flowCode                  14827 non-null  object 
 10  flowDesc                  14827 non-null  int64  
 11  partnerCode               14827 non-null  object 
 12  partnerISO                14827 non-null  object 
 13  partnerDesc               14827 non-null  int64  
 14  partne

Features with only single unique value (nunique = 1) will be removed, as they do not provide any meaningful variance or predictive power for the model.

In [10]:
# Columns with a single unique value or where all values are NaN

columns_single_unique = combined_df_semicond.columns[combined_df_semicond.nunique() < 2]
columns_single_unique

Index(['typeCode', 'refYear', 'flowDesc', 'partnerCode', 'partnerISO',
       'partnerDesc', 'partner2Code', 'partner2ISO', 'classificationCode',
       'classificationSearchCode', 'cmdDesc', 'aggrLevel', 'isLeaf',
       'customsCode', 'customsDesc', 'mosCode', 'motCode', 'grossWgt',
       'isAggregate'],
      dtype='object')

In [11]:
# Drop columns with a single unique value or where all values are NaN

combined_df_semicond.drop(columns=columns_single_unique, inplace=True)

In [12]:
# to ensure no columns with single unique value animore

no_sungle_unique = combined_df_semicond.columns[combined_df_semicond.nunique() < 2]
no_sungle_unique

Index([], dtype='object')

In [13]:
combined_df_semicond.shape  # 19 single unique value columns are dropped

(14827, 28)

In [14]:
combined_df_semicond.columns # to check the columns after dropping

Index(['freqCode', 'refPeriodId', 'refMonth', 'period', 'reporterCode',
       'reporterISO', 'reporterDesc', 'flowCode', 'partner2Desc',
       'isOriginalClassification', 'cmdCode', 'motDesc', 'qtyUnitCode',
       'qtyUnitAbbr', 'qty', 'isQtyEstimated', 'altQtyUnitCode',
       'altQtyUnitAbbr', 'altQty', 'isAltQtyEstimated', 'netWgt',
       'isNetWgtEstimated', 'isGrossWgtEstimated', 'cifvalue', 'fobvalue',
       'primaryValue', 'legacyEstimationFlag', 'isReported'],
      dtype='object')

To get insights into columns to find out if there are redundant columns

In [15]:
combined_df_semicond.head()

,freqCode,refPeriodId,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,partner2Desc,isOriginalClassification,...,altQty,isAltQtyEstimated,netWgt,isNetWgtEstimated,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported
0,20000101,2000,2000,8,ALB,Albania,M,Import,H1,8541,...,False,422.0,True,NaN,3161.0,NaN,3161.0,4,False,False
1,20000101,2000,2000,8,ALB,Albania,X,Export,H1,8541,...,False,8.0,True,NaN,NaN,1086.0,1086.0,4,False,False
2,20000101,2000,2000,12,DZA,Algeria,M,Import,H1,8541,...,False,76542.0,False,NaN,3456956.0,NaN,3456956.0,0,False,False
3,20000101,2000,2000,12,DZA,Algeria,X,Export,H1,8541,...,False,4.0,False,NaN,NaN,1018.0,1018.0,0,False,False
4,20000101,2000,2000,20,AND,Andorra,M,Import,H1,8541,...,False,103.0,False,NaN,12786.0,NaN,12786.0,0,False,False


In [ ]:
# check of countries (debugging)  - I later found out that China has data for a short period of time, so I want to find out if this is the case in the raw dataset.

# Filter and count unique years for China and India
china_years = combined_df_semicond[combined_df_semicond['reporterISO'] == 'China']['refMonth'].nunique()
india_years = combined_df_semicond[combined_df_semicond['reporterISO'] == 'India']['refMonth'].nunique()

# Display results
print(f"China has data for {china_years} years.")
print(f"India has data for {india_years} years.")


China has data for 24 years.
India has data for 25 years.


In this dataset China has data for 24 years. I will check it out in other 2 datasets also.

Some columns are misnamed:
"refMonth", "qtyUnitAbbr", "altQtyUnitAbbr", "isAltQtyEstimated", "netWgt", "isNetWgtEstimated", "isGrossWgtEstimated" and contain not corresponding values. I want to explore values in columns.


In [17]:
# to check unique values in the column "refMonth"

combined_df_semicond.refMonth.unique()  # refMonth is misnamed and contains years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023, 2024], dtype=int64)

In [18]:
combined_df_semicond.qtyUnitAbbr.unique() # contains values, no abbriviatures

array([           nan, 1.68267000e+05, 2.21190000e+05, 1.10766000e+05,
       0.00000000e+00, 5.30000000e+01, 5.35580000e+04, 1.29159300e+06,
       1.49500000e+04, 9.43530000e+04, 1.06000000e+02, 1.22589000e+05,
       2.00000000e+00, 4.87000000e+02, 2.42900000e+03, 1.94622000e+05,
       5.00000000e+00, 1.59400000e+03, 5.12400000e+03, 3.08390000e+04,
       5.44900000e+03, 6.55990000e+04, 1.83450000e+04, 2.64130000e+04,
       3.67600000e+03, 4.15000000e+02, 4.00000000e+00, 2.87800000e+03,
       2.67405000e+05, 9.17800000e+03, 4.31200000e+03, 4.17349000e+05,
       6.04400000e+03, 1.82900000e+03, 7.15900000e+03, 1.00000000e+00,
       5.65000000e+02, 3.00100000e+03, 1.50327000e+05, 2.61949000e+05,
       5.90000000e+01, 6.04000000e+02, 8.21000000e+02, 9.35980000e+04,
       3.80800000e+03, 1.13110000e+05, 1.89720000e+04, 2.53800000e+03,
       6.96330000e+04, 3.18424000e+05, 3.99230000e+04, 3.80000000e+01,
       1.39111000e+05, 1.88000000e+02, 9.00000000e+00, 3.17290000e+04,
      

In [19]:
combined_df_semicond.altQtyUnitAbbr.unique() # contains values, no abbriviatures

array([           nan, 0.00000000e+00, 1.59400000e+03, 4.15000000e+02,
       4.00000000e+00, 1.00000000e+00, 6.04000000e+02, 3.17290000e+04,
       1.80000000e+01, 8.06000000e+02, 1.21000000e+02, 4.59800000e+03,
       3.20000000e+01, 7.61000000e+03, 4.95000000e+02, 3.73000000e+02,
       1.00670000e+04, 8.00000000e+00, 7.00000000e+01, 5.22400000e+03,
       5.30000000e+01, 4.28700000e+03, 5.43000000e+02, 5.70000000e+01,
       1.40000000e+01, 1.14700000e+03, 8.81800000e+03, 5.61916200e+06,
       6.64000000e+02, 1.79000000e+03, 6.36900000e+03, 9.58100000e+03,
       2.40000000e+01, 7.66976000e+05, 1.02522000e+03, 2.55300000e+03,
       2.04700000e+03, 6.00000000e+00, 4.46500000e+03, 5.02170000e+04,
       5.20000000e+01, 2.43605700e+06, 1.00638100e+06, 7.69060000e+02,
       2.00000000e+00, 4.40000000e+01, 1.46727341e+07, 2.30000000e+03,
       1.94040000e+04, 2.10000000e+01, 6.17000000e+02, 2.48519000e+06,
       6.30000000e+00, 2.87049500e+06, 5.87510000e+02, 1.08000000e+03,
      

In [20]:
combined_df_semicond.isAltQtyEstimated.unique() # contains values, no boolean flag

array([4.22000000e+02, 8.00000000e+00, 7.65420000e+04, ...,
       3.27637176e+06, 2.29983581e+06, 1.66592926e+05])

In [21]:
combined_df_semicond.netWgt.unique() # contains boolean flag, no values

array([ True, False])

In [22]:
combined_df_semicond.isNetWgtEstimated.unique() # contains values, no boolean flag

array([           nan, 1.28529000e+05, 0.00000000e+00, 2.58840000e+04,
       7.20000000e+01, 1.10000000e+01, 2.00000000e+01, 1.84660000e+04,
       2.27000000e+02, 6.71420000e+04, 5.30000000e+01, 2.10000000e+01,
       1.83220000e+04, 8.80000000e-01, 5.97190000e+04, 1.97000000e+02,
       1.20000000e+01, 1.70000000e+01, 1.00000000e+00, 1.99730000e+04,
       3.06070000e+02, 1.60000000e+01, 2.90000000e+01, 1.86100000e+03,
       9.06260000e+02, 1.30000000e+01, 9.00000000e+00, 5.55435000e+03,
       1.58910000e+02, 1.00827000e+03, 4.24000000e+00, 9.46500000e+02,
       2.61830000e+02, 1.40000000e+01, 2.00000000e+00, 2.65300000e+01,
       5.23220000e+02, 1.11300000e+02, 5.26880000e+02, 3.10000000e+01,
       3.00000000e+00, 2.56000000e+02, 2.42000000e+02, 3.82800000e+03,
       4.00000000e+00, 1.51083000e+05, 5.29400000e+04, 3.09192000e+05,
       2.16683000e+05, 2.89220000e+04, 6.00000000e+00, 2.42870000e+05,
       1.18323000e+05, 3.11435000e+05, 2.13262000e+05, 5.60300000e+03,
      

In [23]:
combined_df_semicond.isGrossWgtEstimated.unique() # contains values, no boolean flag

array([3.16100000e+03,            nan, 3.45695600e+06, ...,
       4.03864517e+10, 4.21203501e+06, 3.69662129e+07])

In [24]:
combined_df_semicond.isQtyEstimated.value_counts()  # to clarify the values in the column

isQtyEstimated
-1    14361
 5      384
 8       82
Name: count, dtype: int64

In "isQtyEstimated" -1 - Missing values

                     5 - units

                     8 - kg
                    
Only 466 values give information about measurement units. To impute units can mislead, so, this column will be dropped.

* **Necessary columns**:

'refMonth' - Represents years,

"reporterISO" - Identifies country,

"flowCode" - Represents the type of trade flow,

"partner2Desc" - Description of a secondary partner country or geographic area for the respective trade flow,

"cmdCode" - Commodity type, 

"qtyUnitCode" - Contains unit types, only 558 non-values, 

"altQtyUnitCode" - Contains unit types, only 466 non-values,

I will use "qtyUnitCode" and "altQtyUnitCode" to create df_units_filtered['value_per_unit'] for specialized models or sensitivity checks,

"qtyUnitAbbr"(The primary quantity value reported by the country), "altQtyUnitAbbr"(The actual alternative quantity reported by the country) - Due to the fact that I dont use "qtyUnitCode" and "altQtyUnitCode", "qtyUnitAbbr" and "altQtyUnitAbbr" will be used as features — but not for derived calculations,

"isAltQtyEstimated" - Estimated numeric value of the alternative quantity,  

"isNetWgtEstimated" - Estimated net weight value, 

"isGrossWgtEstimated" - Estimated gross weight value, 

"cifvalue" - Includes the cost of goods, freight to the destination and addtionally insurance. I will check it for multicolinearity with "fobvalue".

"fobvalue" - Covers the cost of goods and shipping to the port of origin,

"primaryValue" - A code that indicates: which monetary value field (FOB, CIF, 'Value estimated or derived' or 'Value copied from partner declaration or fallback method') was used   

                to calculate the main reported trade value for that row,

"legacyEstimationFlag" -  A flag indicating whether the data was estimated using an old (legacy) methodology. Can give signal 

"isReported" - A data quality flag:  * Directly reported by the country (True) or *  Estimated or inferred by the database using partner country data or modeling (False). 

* **Columns to delete**:

"freqCode", 'period', 'refPeriodId': Redundant with "refMonth" that represents year,

"reporterCode" - identifies the code of country,

'reporterDesc' - trade flow direction(description of 'flowCode'),

"isOriginalClassification" - Reflects differences in how semiconductor trade data were reported or categorized:  

           1. HS8541 - "Diodes, transistors and similar semiconductor devices; photosensitive semiconductor devices, including photovoltaic cells",
           2. HS8542 - "Electronic integrated circuits and microassemblies",

"motDesc" - indicates type of transportation,

"qty" and "altQty" - identify whether quantity reported or not (mismatch in the name), I have also the column "isReported" to check data is reported or not,

'isQtyEstimated' - Indicates whether the reported quantity (qty) was: Directly reported by the country, or Estimated by the trade database,

netWgt -A Boolean column, identifying is the net weight value available for this trading record or not.

In [ ]:
# unnecessary columns
columns_to_drop = ['freqCode', 'period', 'refPeriodId', 'reporterCode', 'reporterDesc', 'isOriginalClassification','motDesc', 'qty', 'altQty', 'isQtyEstimated','netWgt']

In [26]:
# Drop columns that are not needed for analysis
combined_df_semicond.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# to ensure the columns are dropped
combined_df_semicond.columns                  # now dataset does not contain columns_to_drop            

Index(['refMonth', 'reporterISO', 'flowCode', 'partner2Desc', 'cmdCode',
       'qtyUnitCode', 'qtyUnitAbbr', 'altQtyUnitCode', 'altQtyUnitAbbr',
       'isAltQtyEstimated', 'isNetWgtEstimated', 'isGrossWgtEstimated',
       'cifvalue', 'fobvalue', 'primaryValue', 'legacyEstimationFlag',
       'isReported'],
      dtype='object')

In [28]:
combined_df_semicond.shape  # 11 columns are dropped

(14827, 17)

Some column names are mismatched, so they are needed to be renamed

In [29]:
# to rename the columns for better readability
columns_to_rename = {"refMonth": "Year",
    "reporterISO": "Country",
    "flowCode": "Trade_Flow_Type",
    "partner2Desc": "Partner_Code",
    "cmdCode": "Commodity_Type",
    'qtyUnitCode': 'Reported_Quantity_Unit_Code',
    "qtyUnitAbbr": "Reported_Quantity_Value",
    "altQtyUnitCode": 'Alt_Quantity_Unit_Code',
    "altQtyUnitAbbr": "Alternative_Quantity_Value",
    "isAltQtyEstimated": "Alt_Quantity_Estimated",
    "isNetWgtEstimated": "Net_Weight_Estimated",
    "isGrossWgtEstimated": "Gross_Weight_Estimated",
    "cifvalue": "CIF_Trade_Value_usd",
    "fobvalue": "FOB_Trade_Value_usd",
    "primaryValue": "Trade_Valuation_Source_Code",
    "legacyEstimationFlag": "Legacy_Estimation_Method",
    "isReported": "Officially_Reported_by_Country"    
    }

In [ ]:
# rename mismatched columns
combined_df_semicond.rename(columns=columns_to_rename, inplace=True)

In [ ]:
combined_df_semicond.columns # to check the columns after renaming.  The columns have corresponding names.

Index(['Year', 'Country', 'Trade_Flow_Type', 'Partner_Code', 'Commodity_Type',
       'Reported_Quantity_Unit_Code', 'Reported_Quantity_Value',
       'Alt_Quantity_Unit_Code', 'Alternative_Quantity_Value',
       'Alt_Quantity_Estimated', 'Net_Weight_Estimated',
       'Gross_Weight_Estimated', 'CIF_Trade_Value_usd', 'FOB_Trade_Value_usd',
       'Trade_Valuation_Source_Code', 'Legacy_Estimation_Method',
       'Officially_Reported_by_Country'],
      dtype='object')

In [32]:
# to take a look at the dataset

combined_df_semicond.info()     # There are Missing values in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14827 entries, 0 to 14826
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            14827 non-null  int64  
 1   Country                         14827 non-null  object 
 2   Trade_Flow_Type                 14827 non-null  object 
 3   Partner_Code                    14827 non-null  object 
 4   Commodity_Type                  14827 non-null  object 
 5   Reported_Quantity_Unit_Code     558 non-null    object 
 6   Reported_Quantity_Value         7503 non-null   float64
 7   Alt_Quantity_Unit_Code          466 non-null    object 
 8   Alternative_Quantity_Value      7322 non-null   float64
 9   Alt_Quantity_Estimated          12431 non-null  float64
 10  Net_Weight_Estimated            6402 non-null   float64
 11  Gross_Weight_Estimated          9040 non-null   float64
 12  CIF_Trade_Value_usd             

In this dataset there are values officially reported by country and also estimated. When countries don’t report certain fields, data providers (e.g., UN Comtrade, OECD) estimate values using: Historical ratios, Partner country mirror data, Derived from FOB/CIF... So, estimated values are not high reliable.

To preserve the completeness of the dataset size, estimated values will be included only when no official values are available. Corresponding indicator flags will be created to inform the model that these values may carry a lower level of reliability.

The model will learn to adjust for estimation uncertainty.

In [ ]:
# check the unique values 

combined_df_semicond.Trade_Valuation_Source_Code.unique()  # insights into Trade_Valuation_Source_Code to find out what codes have this column

array([4, 0, 2, 6], dtype=int64)

0 -	CIF value was used (typical for imports)

2 -	Estimated trade value, likely based on modeling or partner data

4 -	FOB value was used (standard for exports and comparisons)

6 -	Value copied from partner's report (e.g., importer didn't report, exporter did)

In [34]:
# Add flag for estimated trade values (Trade_Value_Is_Estimated)

combined_df_semicond['Trade_Value_Is_Estimated'] = combined_df_semicond['Trade_Valuation_Source_Code'] == 2

In [35]:
# Add flag for estimated quantities

combined_df_semicond['Alt_Quantity_Is_Estimated'] = combined_df_semicond['Alternative_Quantity_Value'].isna() & combined_df_semicond['Alt_Quantity_Estimated'].notna()

In [36]:
# Add flag for estimated weights

combined_df_semicond['Net_Weight_Is_Estimated'] = combined_df_semicond['Net_Weight_Estimated'].notna() & (~combined_df_semicond['Officially_Reported_by_Country'])
combined_df_semicond['Gross_Weight_Is_Estimated'] = combined_df_semicond['Gross_Weight_Estimated'].notna() & (~combined_df_semicond['Officially_Reported_by_Country'])

Creating a new feature 'Trade_Value_usd' with a single consistent monetary value using FOB (4), CIF (0) and not reported but still trustable 'Estimated trade value' (2). 

Trade_Valuation_Source_Code (6) means the value was copied from a partner country, valuation unclear, not directly reported. So it must be excluded to avoid inconsistent and noisy data.

In [37]:
combined_df_semicond['Trade_Value_usd'] = np.select(
    [
        combined_df_semicond['Trade_Valuation_Source_Code'] == 4,  # official FOB
        combined_df_semicond['Trade_Valuation_Source_Code'] == 2,  # estimated FOB
        combined_df_semicond['Trade_Valuation_Source_Code'] == 0   # official CIF
    ],
    [
        combined_df_semicond['FOB_Trade_Value_usd'],
        combined_df_semicond['FOB_Trade_Value_usd'],
        combined_df_semicond['CIF_Trade_Value_usd']
    ],
    default=np.nan  # Skip code 6 
)


Columns 'FOB_Trade_Value_usd' and 'CIF_Trade_Value_usd' will not be anymore used, so, they can be dropped.

In [38]:
combined_df_semicond.drop(columns=['FOB_Trade_Value_usd', 'CIF_Trade_Value_usd'], inplace=True)

In [ ]:
# to check the dataset after adding new flags and new Trade_Value_usd column

combined_df_semicond.info()     # Dataset contains new flags and new Trade_Value_usd column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14827 entries, 0 to 14826
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            14827 non-null  int64  
 1   Country                         14827 non-null  object 
 2   Trade_Flow_Type                 14827 non-null  object 
 3   Partner_Code                    14827 non-null  object 
 4   Commodity_Type                  14827 non-null  object 
 5   Reported_Quantity_Unit_Code     558 non-null    object 
 6   Reported_Quantity_Value         7503 non-null   float64
 7   Alt_Quantity_Unit_Code          466 non-null    object 
 8   Alternative_Quantity_Value      7322 non-null   float64
 9   Alt_Quantity_Estimated          12431 non-null  float64
 10  Net_Weight_Estimated            6402 non-null   float64
 11  Gross_Weight_Estimated          9040 non-null   float64
 12  Trade_Valuation_Source_Code     

In [ ]:
# To explore data distribution of tech-leading countries in the dataset over the years. They are key countries in the development of technology

target_countries = ["China", "Germany", "Rep. of Korea", "Japan", "Israel", "USA","India"]

combined_df_semicond[combined_df_semicond["Country"].isin(target_countries)].groupby("Country")[["Year", "Trade_Value_usd"]].agg(["min", "max", "count"])

Year             Trade_Value_usd                    
                min   max count             min           max count
Country                                                            
China          2000  2023    96    1.376574e+09  2.320782e+11    51
Germany        2000  2024   100    1.753443e+09  2.377312e+10    67
India          2000  2024   100    1.244578e+07  6.753888e+09    56
Israel         2000  2024   100    0.000000e+00  5.790731e+09    60
Japan          2000  2024   100    1.373673e+09  3.453208e+10    55
Rep. of Korea  2000  2024   100    7.595841e+08  1.201893e+11    99
USA            2000  2024   100    3.337482e+09  5.413518e+10    55

* **Filling Missing Values**

Merging two columns into one reliable quantity value column:

Alternative estimation column 'Alt_Quantity_Estimated' has values when the main column 'Alternative_Quantity_Value' value is missing. The missing values of main column will be filled by estimated values of alternative estimation column.

In [43]:
combined_df_semicond['Alternative_Quantity_Value'] = combined_df_semicond['Alternative_Quantity_Value'].fillna(
    combined_df_semicond['Alt_Quantity_Estimated']
)

In [44]:
combined_df_semicond.drop(columns=['Alt_Quantity_Estimated'], inplace=True) # to drop the Alt_Quantity_Estimated column as it is not needed anymore. 
                                                                            # It is already merged its values into Alternative_Quantity_Value

In [ ]:
combined_df_semicond.shape  # to check the shape of the DataFrame after dropping the column. It was 20 columns, now it is 19 columns

(14827, 19)

Trade_Value_usd - the actual monetary value of the trade. Not precise values (imputed) distort the entire economic meaning.  It is the main core input. Filling it in with an average or regression would introduce false confidence and harm the validity of analysis. So I will drop NaN values for this column.

'Net_Weight_Estimated' and 'Gross_Weight_Estimated' are physical attributes. Imputing them helps enrich the model, less risk to core logic.

In [46]:
# Impute Net_Weight_Estimated
combined_df_semicond['Net_Weight_Estimated'] = combined_df_semicond.groupby(
    ['Commodity_Type', 'Trade_Flow_Type']
)['Net_Weight_Estimated'].transform(lambda x: x.fillna(x.median()))

# Impute Gross_Weight_Estimated
combined_df_semicond['Gross_Weight_Estimated'] = combined_df_semicond.groupby(
    ['Commodity_Type', 'Trade_Flow_Type']
)['Gross_Weight_Estimated'].transform(lambda x: x.fillna(x.median()))


In [47]:
combined_df_semicond.Net_Weight_Estimated.isna().sum()  # Net_Weight_Estimated has no NaN values anymore

0

In [48]:
combined_df_semicond.Gross_Weight_Estimated.isna().sum()  # Gross_Weight_Estimated has no NaN values anymore

0

In [49]:
# to drop rows with NaN values in Trade_Value_usd column

combined_df_semicond = combined_df_semicond[combined_df_semicond['Trade_Value_usd'].notna()].reset_index(drop=True)

In [ ]:
combined_df_semicond.info()  # Now dataframe has no NaN values in Trade_Value_usd,  Net_Weight_Is_Estimated and  Gross_Weight_Is_Estimated columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9871 entries, 0 to 9870
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            9871 non-null   int64  
 1   Country                         9871 non-null   object 
 2   Trade_Flow_Type                 9871 non-null   object 
 3   Partner_Code                    9871 non-null   object 
 4   Commodity_Type                  9871 non-null   object 
 5   Reported_Quantity_Unit_Code     445 non-null    object 
 6   Reported_Quantity_Value         5073 non-null   float64
 7   Alt_Quantity_Unit_Code          432 non-null    object 
 8   Alternative_Quantity_Value      9305 non-null   float64
 9   Net_Weight_Estimated            9871 non-null   float64
 10  Gross_Weight_Estimated          9871 non-null   float64
 11  Trade_Valuation_Source_Code     9871 non-null   int64  
 12  Legacy_Estimation_Method        98

In [56]:
combined_df_semicond.duplicated().sum()  # there are no duplicate rows to drop

0

In [57]:
# check of countries (debugging)

# Filter and count unique years for China and India
china_years_s = combined_df_semicond[combined_df_semicond['Country'] == 'China']['Year'].nunique()
india_years_s = combined_df_semicond[combined_df_semicond['Country'] == 'India']['Year'].nunique()
neth_years_s = combined_df_semicond[combined_df_semicond['Country'] == 'Netherlands']['Year'].nunique()
tai_years_s = combined_df_semicond[combined_df_semicond['Country'] == 'Taiwan']['Year'].nunique()
sing_years_s = combined_df_semicond[combined_df_semicond['Country'] == 'Singapore']['Year'].nunique()


# Display results
print(f"China has data for {china_years_s} years.")
print(f"India has data for {india_years_s} years.")
print(f"Netherlands has data for {neth_years_s} years.")
print(f"Taiwan has data for {tai_years_s} years.")
print(f"Singapore has data for {sing_years_s} years.")

China has data for 17 years.
India has data for 22 years.
Netherlands has data for 15 years.
Taiwan has data for 0 years.
Singapore has data for 21 years.


In [ ]:
#I have commented this line to avoid saving the file again, as it is already saved

# combined_df_semicond.to_csv("../Data/Processed/df_semicond_final.csv", index=False)  # Save the "combined_df_semicond" DataFrame to a CSV file

**Preprocessing Employment by Occupation dataset before merging**

The dataset EMP_TEMP_SEX_OCU_NB_A-full-2025-05-06.csv represents employment statistics by occupation, sex, and country over time

In [ ]:
# loading the Employment dataset

path_emp = "../Data/Row/EMP_TEMP_SEX_OCU_NB_A-full-2025-05-06.csv"
df_emp_base = pd.read_csv(path_emp, encoding='latin1')

In [60]:
df_emp = df_emp_base.copy() # to avoid modifying the original dataframe

In [ ]:
# Filter and count unique years for China and India (debugging)
china_years = df_emp[df_emp['ref_area.label'] == 'China']['time'].nunique()
india_years = df_emp[df_emp['ref_area.label'] == 'India']['time'].nunique() 
neth_years = df_emp[df_emp['ref_area.label'] == 'Netherlands']['time'].nunique()
tai_years = df_emp[df_emp['ref_area.label'] == 'Taiwan, China']['time'].nunique()
sing_years = df_emp[df_emp['ref_area.label'] == 'Singapore']['time'].nunique()
# Display results
print(f"China has data for {china_years} years.")
print(f"India has data for {india_years} years.")
print(f"Netherlands has data for {neth_years} years.")
print(f"Taiwan has data for {tai_years} years.")
print(f"Singapore has data for {sing_years} years.")

China has data for 2 years.
India has data for 12 years.
Netherlands has data for 43 years.
Taiwan has data for 20 years.
Singapore has data for 50 years.


China plays a central role in the focus of my project study. Employment is the target of my model and the dataset contains data for China only for 2 years. 

The model has almost no way to "learn" China's specific patterns. Any conclusions about China might be overconfident or unstable.

In [ ]:
# to check the first few rows of the dataframe

df_emp.head()

,"ï»¿""ref_area""",ref_area.label,source,source.label,indicator,indicator.label,sex,sex.label,classif1,classif1.label,time,obs_value,obs_status,obs_status.label,note_classif,note_classif.label,note_indicator,note_indicator.label,note_source,note_source.label
0,ABW,Aruba,BA:829,LFS - Labour Force Survey,EMP_TEMP_SEX_OCU_NB,Employment by sex and occupation (thousands),SEX_T,Total,OCU_SKILL_TOTAL,Occupation (Skill level): Total,2011,47.943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,Aruba,BA:829,LFS - Labour Force Survey,EMP_TEMP_SEX_OCU_NB,Employment by sex and occupation (thousands),SEX_T,Total,OCU_SKILL_L3-4,Occupation (Skill level): Skill levels 3 and 4...,2011,14.854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABW,Aruba,BA:829,LFS - Labour Force Survey,EMP_TEMP_SEX_OCU_NB,Employment by sex and occupation (thousands),SEX_T,Total,OCU_SKILL_L2,Occupation (Skill level): Skill level 2 ~ medium,2011,23.618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABW,Aruba,BA:829,LFS - Labour Force Survey,EMP_TEMP_SEX_OCU_NB,Employment by sex and occupation (thousands),SEX_T,Total,OCU_SKILL_L1,Occupation (Skill level): Skill level 1 ~ low,2011,9.208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABW,Aruba,BA:829,LFS - Labour Force Survey,EMP_TEMP_SEX_OCU_NB,Employment by sex and occupation (thousands),SEX_T,Total,OCU_SKILL_X,Occupation (Skill level): Not elsewhere classi...,2011,0.263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# to get a concise summary of the DataFrame

df_emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161884 entries, 0 to 161883
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ï»¿"ref_area"         161884 non-null  object 
 1   ref_area.label        161884 non-null  object 
 2   source                161884 non-null  object 
 3   source.label          161884 non-null  object 
 4   indicator             161884 non-null  object 
 5   indicator.label       161884 non-null  object 
 6   sex                   161884 non-null  object 
 7   sex.label             161884 non-null  object 
 8   classif1              161884 non-null  object 
 9   classif1.label        161884 non-null  object 
 10  time                  161884 non-null  int64  
 11  obs_value             159642 non-null  float64
 12  obs_status            13426 non-null   object 
 13  obs_status.label      13426 non-null   object 
 14  note_classif          1010 non-null    object 
 15  

In [64]:
df_emp.duplicated().sum()   # there are no duplicate rows in the DataFrame

0

* not to use

`ref_area` - Country code                                       
`ref_area.label` - Country name
`sex` - Code for sex (`SEX_T`, `SEX_M`, `SEX_F`)                    
`sex.label` - Full label (`Total`, `Male`, `Female`)                           
`classif1` - Occupation code (ISCO level or skill level)
`classif1.label` - Occupation or skill level label                                  
`time` - Year 
"obs_value" - Number of employed persons - key metric for modeling             
"obs_status" and all `note_...` columns - Metadata               


In [65]:
# to remove unwanted characters from the column names and rename them

df_emp.rename(columns=lambda x: x.replace("ï»¿", ""), inplace=True)   

In [66]:
df_emp.columns   # unwanted characters removed and column ï»¿"ref_area"  renamed as "ref_area"

Index(['"ref_area"', 'ref_area.label', 'source', 'source.label', 'indicator',
       'indicator.label', 'sex', 'sex.label', 'classif1', 'classif1.label',
       'time', 'obs_value', 'obs_status', 'obs_status.label', 'note_classif',
       'note_classif.label', 'note_indicator', 'note_indicator.label',
       'note_source', 'note_source.label'],
      dtype='object')

Features with only single unique value (nunique = 1) will be removed, as they do not provide any meaningful variance or predictive power for the model.

In [67]:
# Columns with a single unique value or where all values are NaN

single_unique_emp = df_emp.columns[df_emp.nunique() < 2]
single_unique_emp

Index(['indicator', 'indicator.label'], dtype='object')

These columns contain codes and there are according columns with '.label' those are more interpretable than columns with codes. So, columns with ".label" will be  used, columns with code will be dropped:

"ref_area"', "source", "sex",  "classif1", "obs_status", "note_classif", "note_indicator", "note_source"

In [68]:
# list of columns with codes are not needed for analysis

columns_emp_to_drop = ['"ref_area"', "source", "sex",  "classif1", "obs_status", "note_classif", "note_indicator", "note_source"]+ list(single_unique_emp)

In [69]:
# Dropping list of columns are not needed for analysis

df_emp.drop(columns=columns_emp_to_drop, inplace=True)

In [70]:
no_sungle_unique_emp = df_emp.columns[df_emp.nunique() < 2]    # there are no columns with single unique value animore
no_sungle_unique_emp

Index([], dtype='object')

In [71]:
# to check what columns are left after dropping

df_emp.columns   # columns_emp_to_drop is dropped

Index(['ref_area.label', 'source.label', 'sex.label', 'classif1.label', 'time',
       'obs_value', 'obs_status.label', 'note_classif.label',
       'note_indicator.label', 'note_source.label'],
      dtype='object')

In [72]:
df_emp.shape  # 10 columns are dropped

(161884, 10)

In [ ]:
# To find out data distribution of tech-leading countries in the dataset over the years

target_countries = ["China", "Germany", "Republic of Korea", "Japan", "Israel", "United States of America","India"]

df_emp[df_emp["ref_area.label"].isin(target_countries)].groupby("ref_area.label")[["time", "obs_value"]].agg(["min", "max", "count"])

# China has data in 2000 - 2005 years for 2 years, other countries have data untill 2023-24 years

time             obs_value                  
                           min   max count       min         max count
ref_area.label                                                        
China                     2000  2005    72  1517.500  668748.890    72
Germany                   1992  2024  1680     1.229   43262.500  1676
India                     1994  2024   550     7.684  482793.235   508
Israel                    1970  2023  2127     0.800    4309.320  2127
Japan                     1970  2024  1929    30.000   67810.000  1929
Republic of Korea         1970  2024  2115     1.000   28695.214  2115
United States of America  1970  2024  1740    80.292  161345.530  1740

In [74]:
# Filter and count unique years for China and India
china_years = df_emp[df_emp['ref_area.label'] == 'China']['time'].nunique()
india_years = df_emp[df_emp['ref_area.label'] == 'India']['time'].nunique() 
neth_years = df_emp[df_emp['ref_area.label'] == 'Netherlands']['time'].nunique()
tai_years = df_emp[df_emp['ref_area.label'] == 'Taiwan, China']['time'].nunique()
sing_years = df_emp[df_emp['ref_area.label'] == 'Singapore']['time'].nunique()
# Display results
print(f"China has data for {china_years} years.")
print(f"India has data for {india_years} years.")
print(f"Netherlands has data for {neth_years} years.")
print(f"Singapore has data for {sing_years} years.")
print(f"Taiwan has data for {tai_years} years.")

China has data for 2 years.
India has data for 12 years.
Netherlands has data for 43 years.
Singapore has data for 50 years.
Taiwan has data for 20 years.


In [75]:
# Filter df_emp for China and India, and show available years(debugging)
# Force full output of years for China and India
years_dict = df_emp[df_emp['ref_area.label'].isin(['China', 'India','Netherlands', 'Singapore', 'Taiwan, China'])] \
    .groupby('ref_area.label')['time'] \
    .unique() \
    .apply(sorted) \
    .to_dict()

# Print all years for each country
for country, years in years_dict.items():
    print(f"{country}: {years}")

China: [2000, 2005]
India: [1994, 2000, 2005, 2010, 2012, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Netherlands: [1977, 1979, 1981, 1983, 1985, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Singapore: [1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Taiwan, China: [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


China's limited data range (2000-2005) may underestimate its employment growth and miss later impacts of semiconductor adoption (e.g., automation-driven displacement), which accelerated post-2005. This gap could bias the analysis compared to other countries with data until 2023-24. 

In [ ]:
# Filter India and exclude year 1994, then count rows (debugging of key countries - whether they have enough data)
india_count = df_emp[
    (df_emp['ref_area.label'] == 'India') &
    (df_emp['time'] != 1994)
].shape[0]

print(f"India has {india_count} records excluding 1994.")

India has 502 records excluding 1994.


To ensure data integrity and meaningful analysis of employment trends, I excluded records labeled as ‘Unreliable’—which indicate insufficient sample sizes or inconsistent reporting—and ‘Break in series’, which reflect methodological changes that disrupt comparability over time.

df_emp['obs_status.label'].isna()

To make sure analyzing real employment trends, I filtered the employment data to include only records derived from Labour Force Surveys. These are internationally standardized and capture occupation-level trends with the methodological rigor required to model global labor shifts.

df_emp['source.label'].str.contains('Labour Force Survey', na=False)

The note_classif.label column identifies observations with partial or irregular occupation groups.

I excluded employment observations flagged with nonstandard occupational coverage (note_classif.label) to ensure the dataset reflects clearly defined, internationally comparable occupation categories. This improves the accuracy and interpretability of downstream modeling.

In [77]:
df_emp['source.label'].unique()  # to check unique values in the 'source.label' column

array(['LFS - Labour Force Survey', 'PC - Population and Housing Census',
       'HIES - Households Living Conditions Survey',
       'LFS - Employment Survey', 'PC - Population Census',
       'HIES - Core Welfare Indicators Questionnaire',
       'HIES - Living Standards Survey', 'OE - Official Estimates',
       'LFS - Permanent Household Survey, Urban',
       'LFS - Household Labour Force Survey',
       'LFS - EU Labour Force Survey',
       'HIES - Monitoring Survey of the Modular and Integrated Survey of Household Living Conditions',
       'PC - Population projections and Revised Studies on Social Demand 2002-2030',
       'ADM-IR - Returns and Censuses of staff of ministerial departments, the Social Security Office, the Computer Office and the periodic returns on the Workforce',
       'LFS - Regional Integrated Survey on Employment and the Informal Sector',
       'HIES - Harmonized Survey on Household Living Conditions',
       'ADM-IR - Insurance Records, other',
       'O

In [79]:
# Use partial keyword filtering instead of strict full match
keyword_filters = [
    'Labour Force',
    'Employment',
    'Household Employment',
    'Official Estimates',
    'Population Census',
    'Current Population Survey',
    'Economically Active Population Survey',
    'LFS - Periodic Labour Force Survey',
    'HS - National Sample Survey'
]       

df_emp = df_emp[
    df_emp['obs_status.label'].isna() &
    df_emp['source.label'].apply(lambda x: any(keyword in x for keyword in keyword_filters)) &
    df_emp['note_classif.label'].isna()
]


In [81]:
# Filter India and exclude year 1994, then count rows(debugging)
india_count = df_emp[
    (df_emp['ref_area.label'] == 'India') &
    (df_emp['time'] != 1994)
].shape[0]

print(f"India has {india_count} records excluding 1994.")

India has 445 records excluding 1994.


In [ ]:
# to check unique values in the 'ref_area.label' column.

df_emp["ref_area.label"].nunique() # Dataset contains data for 170 countries.

170

In [ ]:
target_countries = ['China', 'United States of America', 'Republic of Korea', 'Israel', 'Japan', 'Germany', 'India',  'Netherlands', 'Taiwan, China', 'Singapore']  
df_emp[df_emp["ref_area.label"].isin(target_countries)]["ref_area.label"].unique()  # The top 9 countries in the global semiconductor industry are represented in the dataset.


array(['China', 'Germany', 'India', 'Israel', 'Japan',
       'Republic of Korea', 'Netherlands', 'Singapore', 'Taiwan, China',
       'United States of America'], dtype=object)

'note_indicator.label' - Mostly indicates breaks in methodology (which I've already handled), adds no modeling value, contains mostly: "Break in series", 
                                                                "Working time coverage..."

'note_source.label' - Reference and sample notes; no modeling value.

So they will be dropped.

In [84]:
columns_emp_to_drop_2 = ['source.label', 'obs_status.label', 'note_classif.label', 'note_indicator.label', 'note_source.label']

In [85]:
df_emp.drop(columns=columns_emp_to_drop_2, inplace=True)

In [86]:
# reset index after dropping rows

df_emp.reset_index(drop=True, inplace=True) 

In [87]:
df_emp.info() # "columns_emp_to_drop_2" is dropped and index is reset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119064 entries, 0 to 119063
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ref_area.label  119064 non-null  object 
 1   sex.label       119064 non-null  object 
 2   classif1.label  119064 non-null  object 
 3   time            119064 non-null  int64  
 4   obs_value       119064 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.5+ MB


In [88]:
# columns are needed to be renamed for better readability and the ability to use them to merge with other datasets

columns_emp_to_rename = {'ref_area.label': 'Country',
                        'sex.label': 'Gender',
                        'classif1.label': 'Occupation',
                        'time': 'Year',
                        'obs_value': 'Employment_Count'}

In [89]:
# renaming columns 

df_emp.rename(columns=columns_emp_to_rename, inplace=True)

In [90]:
df_emp.info() # columns are renamed; No missing values in the DataFrame. It is ready to be merged with other datasets

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119064 entries, 0 to 119063
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Country           119064 non-null  object 
 1   Gender            119064 non-null  object 
 2   Occupation        119064 non-null  object 
 3   Year              119064 non-null  int64  
 4   Employment_Count  119064 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.5+ MB


In [ ]:
df_emp.Year.unique()  # to check unique values in the 'Year' column. There are data for 1969-2023 years.

array([2011, 2007, 1997, 1994, 2021, 2022, 2014, 2010, 2001, 1992, 1984,
       2023, 2020, 2019, 2018, 2017, 2016, 2015, 2013, 2012, 2009, 2008,
       2006, 2005, 2004, 2003, 2002, 2000, 1998, 1996, 1995, 1993, 1989,
       1988, 1976, 1975, 1974, 1973, 2024, 1999, 1991, 1990, 1987, 1986,
       1985, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1972, 1971, 1970,
       1969], dtype=int64)

Technology has significantly developed after the year 2000. Therefore, the model will be trained on data from 2000 onwards.

In [ ]:
df_emp = df_emp[df_emp['Year'].between(2000, 2023)] # filtering data for years 2000-2023

In [ ]:
# to check unique values in the 'Year' column.

df_emp.Year.unique()  # df_emp filtered for years 2000-2023

array([2011, 2007, 2021, 2022, 2014, 2010, 2001, 2023, 2020, 2019, 2018,
       2017, 2016, 2015, 2013, 2012, 2009, 2008, 2006, 2005, 2004, 2003,
       2002, 2000], dtype=int64)

In [ ]:
df_emp.Year.nunique() # column contains 24 years

24

In [ ]:
df_emp.info() # to check dataset before saving to csv file

<class 'pandas.core.frame.DataFrame'>
Index: 84426 entries, 0 to 119063
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           84426 non-null  object 
 1   Gender            84426 non-null  object 
 2   Occupation        84426 non-null  object 
 3   Year              84426 non-null  int64  
 4   Employment_Count  84426 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 3.9+ MB


In [ ]:
#I have commented this line to avoid saving the file again, as it is already saved 

# df_emp.to_csv("../Data/Processed/df_emp_final.csv", index=False)  

**Preprocessing "Research and development expenditure (% of GDP)" dataset before merging**

The dataset API_GB.XPD.RSDV.GD.ZS_DS2_World_RD is from the World Bank and it represents:

"Research and development expenditure (% of GDP)"

This dataset shows how much each country spends on R&D as a percentage of its gross domestic product (GDP), annually.

In [ ]:
# loading the dataset

path_RD = "../Data/Row/API_GB.XPD.RSDV.GD.ZS_DS2_World_RD.csv"
df_RD_base = pd.read_csv(path_RD, skiprows=3)

In [99]:
df_RD = df_RD_base.copy() # to avoid modifying the original dataframe

In [100]:
# to check the first few rows of the dataframe

df_RD.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# to get a concise summary of the DataFrame

df_RD.info()      

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 70 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1960            0 non-null      float64
 5   1961            0 non-null      float64
 6   1962            0 non-null      float64
 7   1963            0 non-null      float64
 8   1964            0 non-null      float64
 9   1965            0 non-null      float64
 10  1966            0 non-null      float64
 11  1967            0 non-null      float64
 12  1968            0 non-null      float64
 13  1969            0 non-null      float64
 14  1970            0 non-null      float64
 15  1971            0 non-null      float64
 16  1972            0 non-null      float64
 17  1973            0 non-null      flo

In the dataframe there are columns with country names and years, and the values are the research and development expenditure as a percentage of GDP.

Each year is represented as a separate column. The dataset has Wide format.

Features with only single unique value (nunique = 1) will be removed, as they do not provide any meaningful variance or predictive power for the model.

In [102]:
# Columns with a single unique value or where all values are NaN

single_unique_RD = df_RD.columns[df_RD.nunique() < 2]
single_unique_RD

Index(['Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963',
       '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972',
       '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981',
       '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990',
       '1991', '1992', '1993', '1994', '1995', '2024', 'Unnamed: 69'],
      dtype='object')

In [103]:
# Dropping Single unique columns

df_RD.drop(columns=single_unique_RD, inplace=True)

In [104]:
df_RD.columns # Single Unique columns are dropped   # For later merging I do not need years 1996-1999 and 'Country Code (I need 'Country Name' for merging). 
                # They will be dropped 

Index(['Country Name', 'Country Code', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023'],
      dtype='object')

In [105]:
unusable_columns_RD = ['1996', '1997', '1998', '1999', 'Country Code']
df_RD.drop(columns=unusable_columns_RD, inplace=True)
df_RD.columns                               # unusable years  and 'Country Code' are dropped. Now the dataframe contains only years from 2000 to 2023 and the country names

Index(['Country Name', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'],
      dtype='object')

In [106]:
df_RD.duplicated().sum()  # There are no duplicate rows in the dataframe

0

In [107]:
df_RD.info()  # Now df_RD oOnly includes years 2000–2023 and retains only "Country Name" + numeric columns for R&D expenditure (% of GDP)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  266 non-null    object 
 1   2000          99 non-null     float64
 2   2001          108 non-null    float64
 3   2002          122 non-null    float64
 4   2003          121 non-null    float64
 5   2004          121 non-null    float64
 6   2005          120 non-null    float64
 7   2006          111 non-null    float64
 8   2007          130 non-null    float64
 9   2008          123 non-null    float64
 10  2009          121 non-null    float64
 11  2010          123 non-null    float64
 12  2011          120 non-null    float64
 13  2012          112 non-null    float64
 14  2013          122 non-null    float64
 15  2014          117 non-null    float64
 16  2015          131 non-null    float64
 17  2016          118 non-null    float64
 18  2017          124 non-null    

The dataset is in wide format. It is needed to be reshaped from wide to long format for easier merging and analysis.

In [108]:
df_RD_long = df_RD.melt(     
    id_vars="Country Name",
    var_name="Year",
    value_name="RD_Expenditure_GDP_Percent"                         # Melt the DataFrame to long format
)                       

df_RD_long["Year"] = df_RD_long["Year"].astype(int)                 # Convert 'Year' to integer type for consistency


In [109]:
# Renaming the 'Country Name' column to 'Country' for consistency with other datasets

df_RD_long.rename(columns={'Country Name': 'Country'}, inplace=True)

In [110]:
df_RD_long.head()  # The DataFrame is now in long format with columns: "Country", "Year", and "RD_Expenditure_GDP_Percent"

,Country,Year,RD_Expenditure_GDP_Percent
0,Aruba,2000,NaN
1,Africa Eastern and Southern,2000,NaN
2,Afghanistan,2000,NaN
3,Africa Western and Central,2000,NaN
4,Angola,2000,NaN


In [111]:
# To get info about data distribution of leading countries in the dataset over the years

target_countries = ["China", "Germany", "Korea, Rep.", "Japan", "Israel", "United States","India", 'Netherlands', 'Taiwan', 'Singapore']

df_RD_long[df_RD_long["Country"].isin(target_countries)].groupby("Country")[["Year", "RD_Expenditure_GDP_Percent"]].agg(["min", "max", "count"])

Year             RD_Expenditure_GDP_Percent               
                min   max count                        min      max count
Country                                                                  
China          2000  2023    24                    0.89316  2.55518    23
Germany        2000  2023    24                    2.40437  3.16701    23
India          2000  2023    24                    0.64558  0.85876    21
Israel         2000  2023    24                    3.74903  6.01924    23
Japan          2000  2023    24                    2.85841  3.40537    23
Korea, Rep.    2000  2023    24                    2.12519  5.21081    23
Netherlands    2000  2023    24                    1.62269  2.32182    23
Singapore      2000  2023    24                    1.80947  2.59674    21
United States  2000  2023    24                    2.48686  3.58623    23

The leading countries in the dataset have comprehensive data coverage spanning the years 2000 to 2023.

In [112]:
df_RD_long.isna().sum()  # Checking for NaN values in the DataFrame 

Country                          0
Year                             0
RD_Expenditure_GDP_Percent    3657
dtype: int64

I retained missing values in the "RD_Expenditure_GDP_Percent" feature to reflect the absence of reported R&D activity in specific country-years. These Missing values are signals of technological lag, not a data flaw. This helps the model distinguish between innovation-intensive and lagging economies. 

I will build my model on Tree-based models (like XGBoost, RandomForest) those will handle missing R&D smartly.

In [113]:
# last check of the DataFrame before saving
df_RD_long.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6384 entries, 0 to 6383
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Country                     6384 non-null   object 
 1   Year                        6384 non-null   int32  
 2   RD_Expenditure_GDP_Percent  2727 non-null   float64
dtypes: float64(1), int32(1), object(1)
memory usage: 124.8+ KB


In [ ]:
#I have commented this line to avoid saving the file again, as it is already saved

# Saving the cleaned DataFrame to a CSV file

# df_RD_long.to_csv("../Data/Processed/df_RD_final.csv", index=False)

**Merging 3 cleaned datasets**

Loading the preprocessed and saved as csv files 3 datasets those will be merged for modeling

In [ ]:
path_semicond = "../Data/Processed/df_semicond_final.csv"
path_emp = "../Data/Processed/df_emp_final.csv"
path_RD = "../Data/Processed/df_RD_final.csv"
df_semicond_to_merge= pd.read_csv(path_semicond)
df_emp_to_merge = pd.read_csv(path_emp)
df_RD_to_merge = pd.read_csv(path_RD)

To be sure 3 datasets are ready for merging I am checking them again

In [117]:
df_semicond_to_merge.head()  # to check the first few rows of the dataframe

,Year,Country,Trade_Flow_Type,Partner_Code,Commodity_Type,Alternative_Quantity_Value,Net_Weight_Estimated,Gross_Weight_Estimated,Trade_Valuation_Source_Code,Legacy_Estimation_Method,Officially_Reported_by_Country,Trade_Value_Is_Estimated,Alt_Quantity_Is_Estimated,Net_Weight_Is_Estimated,Gross_Weight_Is_Estimated,Trade_Value_usd
0,2000,Albania,Import,H1,"Diodes, transistors, similar semiconductor dev...",422.0,0.0,3161.0,4,False,False,False,True,False,True,3161.0
1,2000,Albania,Export,H1,"Diodes, transistors, similar semiconductor dev...",8.0,0.0,0.0,4,False,False,False,True,False,False,1086.0
2,2000,Algeria,Export,H1,"Diodes, transistors, similar semiconductor dev...",4.0,0.0,0.0,0,False,False,False,True,False,False,1018.0
3,2000,Antigua and Barbuda,Export,H1,"Diodes, transistors, similar semiconductor dev...",40.0,0.0,0.0,0,False,False,False,True,False,False,204.0
4,2000,Azerbaijan,Import,H1,"Diodes, transistors, similar semiconductor dev...",124.0,0.0,21496.0,2,False,False,True,True,False,True,21496.0


In [118]:
df_semicond_to_merge.info()  # to get a concise summary of the DataFrame 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9871 entries, 0 to 9870
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            9871 non-null   int64  
 1   Country                         9871 non-null   object 
 2   Trade_Flow_Type                 9871 non-null   object 
 3   Partner_Code                    9871 non-null   object 
 4   Commodity_Type                  9871 non-null   object 
 5   Alternative_Quantity_Value      9305 non-null   float64
 6   Net_Weight_Estimated            9871 non-null   float64
 7   Gross_Weight_Estimated          9871 non-null   float64
 8   Trade_Valuation_Source_Code     9871 non-null   int64  
 9   Legacy_Estimation_Method        9871 non-null   bool   
 10  Officially_Reported_by_Country  9871 non-null   bool   
 11  Trade_Value_Is_Estimated        9871 non-null   bool   
 12  Alt_Quantity_Is_Estimated       98

In [119]:
df_emp_to_merge.head()  # to check the first few rows of the dataframe

,Country,Gender,Occupation,Year,Employment_Count
0,Aruba,Total,Occupation (Skill level): Total,2011,47.943
1,Aruba,Total,Occupation (Skill level): Skill levels 3 and 4...,2011,14.854
2,Aruba,Total,Occupation (Skill level): Skill level 2 ~ medium,2011,23.618
3,Aruba,Total,Occupation (Skill level): Skill level 1 ~ low,2011,9.208
4,Aruba,Total,Occupation (Skill level): Not elsewhere classi...,2011,0.263


In [120]:
df_emp_to_merge.info()  # to get a concise summary of the DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84426 entries, 0 to 84425
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           84426 non-null  object 
 1   Gender            84426 non-null  object 
 2   Occupation        84426 non-null  object 
 3   Year              84426 non-null  int64  
 4   Employment_Count  84426 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 3.2+ MB


In [121]:
df_RD_to_merge.head()  # to check the first few rows of the dataframe

,Country,Year,RD_Expenditure_GDP_Percent
0,Aruba,2000,NaN
1,Africa Eastern and Southern,2000,NaN
2,Afghanistan,2000,NaN
3,Africa Western and Central,2000,NaN
4,Angola,2000,NaN


In [122]:
df_RD_to_merge.info()  # to get a concise summary of the DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6384 entries, 0 to 6383
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Country                     6384 non-null   object 
 1   Year                        6384 non-null   int64  
 2   RD_Expenditure_GDP_Percent  2727 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 149.8+ KB


* **Country names will be validated and standardized to maintain consistency across datasets before merging**

In [ ]:
# Extract unique country names from each dataset
trade_countries = set(df_semicond_to_merge['Country'].unique())
emp_countries   = set(df_emp_to_merge['Country'].unique())
rd_countries    = set(df_RD_to_merge['Country'].unique())

In [ ]:
# Compare them pairwise
only_in_trade = trade_countries - emp_countries - rd_countries
only_in_emp   = emp_countries - trade_countries - rd_countries
only_in_rd    = rd_countries - trade_countries - emp_countries

# Print results
print("🔹 Countries only in Trade dataset:")
print(sorted(only_in_trade))

print("\n🔹 Countries only in Employment dataset:")
print(sorted(only_in_emp))

print("\n🔹 Countries only in R&D dataset:")
print(sorted(only_in_rd))

# Show all countries that don't match in all 3
all_countries = trade_countries.union(emp_countries).union(rd_countries)
in_all_three = trade_countries & emp_countries & rd_countries
mismatched_countries = all_countries - in_all_three

print("\n❗ Countries NOT present in all three datasets:")
print(sorted(mismatched_countries))


🔹 Countries only in Trade dataset:
['Bosnia Herzegovina', 'Cayman Isds', 'Central African Rep.', 'China, Hong Kong SAR', 'China, Macao SAR', 'Congo', 'Cook Isds', 'Curaçao', "Côte d'Ivoire", 'Dem. Rep. of the Congo', 'Dominican Rep.', 'FS Micronesia', 'Faeroe Isds', 'Iran', "Lao People's Dem. Rep.", 'Mayotte (Overseas France)', 'Netherlands Antilles (...2010)', 'Other Asia, nes', 'Rep. of Korea', 'Rep. of Moldova', 'Saint Kitts and Nevis', 'Serbia and Montenegro (...2005)', 'Solomon Isds', 'State of Palestine', 'Sudan (...2011)', 'Syria', 'Turks and Caicos Isds', 'Türkiye', 'USA', 'United Rep. of Tanzania', 'Venezuela', 'Wallis and Futuna Isds']

🔹 Countries only in Employment dataset:
['Anguilla', 'Cook Islands', 'CuraÃ§ao', "CÃ´te d'Ivoire", 'Iran (Islamic Republic of)', 'Macao, China', 'Netherlands Antilles', 'Niue', 'Occupied Palestinian Territory', 'Republic of Korea', 'Republic of Moldova', 'RÃ©union', 'Taiwan, China', 'Tokelau', 'TÃ¼rkiye', 'United Kingdom of Great Britain and N

Plenty of countries have mismatched names. They should be renamed to a common naming

In [ ]:
# Extract unique country names from each dataset
trade_countries = set(df_semicond_to_merge['Country'].unique())
emp_countries   = set(df_emp_to_merge['Country'].unique())
rd_countries    = set(df_RD_to_merge['Country'].unique())

# Define standardized country corrections to resolve mismatches
country_corrections = {
    'Bahamas, The': 'Bahamas',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Bosnia Herzegovina': 'Bosnia and Herzegovina',
    'Cayman Isds': 'Cayman Islands',
    'Central African Rep.': 'Central African Republic',
    'China, Hong Kong SAR': 'Hong Kong SAR, China',
    'China, Macao SAR': 'Macao SAR, China',
    'Congo': 'Congo, Rep.',
    'Cook Isds': 'Cook Islands',
    'CuraÃ§ao': 'Curacao',
    'Curaçao': 'Curacao',
    "Côte d'Ivoire": "Cote d'Ivoire",
    'Dem. Rep. of the Congo': 'Congo, Dem. Rep.',
    'Dominican Rep.': 'Dominican Republic',
    'Egypt': 'Egypt, Arab Rep.',
    'FS Micronesia': 'Micronesia, Fed. Sts.',
    'Faeroe Isds': 'Faroe Islands',
    'Iran': 'Iran, Islamic Rep.',
    'Iran (Islamic Republic of)': 'Iran, Islamic Rep.',
    'Kyrgyzstan': 'Kyrgyz Republic',
    "Lao People's Dem. Rep.": 'Lao PDR',
    'Mayotte (Overseas France)': 'Mayotte',
    'Netherlands Antilles (...2010)': 'Netherlands Antilles',
    'Occupied Palestinian Territory': 'West Bank and Gaza',
    'Rep. of Korea': 'Korea, Rep.',
    'Republic of Korea': 'Korea, Rep.',
    'Republic of Moldova': 'Moldova',
    'Rep. of Moldova': 'Moldova',
    'RÃ©union': 'Reunion',
    'Saint Kitts and Nevis': 'St. Kitts and Nevis',
    'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
    'Serbia and Montenegro (...2005)': 'Serbia',
    'Solomon Isds': 'Solomon Islands',
    'State of Palestine': 'West Bank and Gaza',
    'Sudan (...2011)': 'Sudan',
    'Syria': 'Syrian Arab Republic',
    'Turks and Caicos Isds': 'Turks and Caicos Islands',
    'Türkiye': 'Turkiye',
    'TÃ¼rkiye': 'Turkiye',
    'USA': 'United States of America',
    'United Rep. of Tanzania': 'Tanzania',
    'Venezuela, RB': 'Venezuela',
    'Wallis and Futuna Isds': 'Wallis and Futuna',
    'Yemen, Rep.': 'Yemen',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Wallis and Futuna': 'Wallis and Futuna Isds.',
    'Réunion': 'Reunion', 
    "Korea, Dem. People's Rep.": 'Korea, Dem. Rep.',
    'Gambia': 'Gambia, The',
    'Slovakia': 'Slovak Republic',
    'United States': 'United States of America',
    'United States, The': 'United States of America',
    'Gambia, ': 'Gambia, The'
}

In [ ]:
# Apply correctioned country names to each DataFrame
for df in [df_semicond_to_merge, df_emp_to_merge, df_RD_to_merge]:
    df['Country'] = df['Country'].replace(country_corrections)

# Compare them pairwise
trade_countries = set(df_semicond_to_merge['Country'].unique())
emp_countries   = set(df_emp_to_merge['Country'].unique())
rd_countries    = set(df_RD_to_merge['Country'].unique())

only_in_trade = trade_countries - emp_countries - rd_countries
only_in_emp   = emp_countries - trade_countries - rd_countries
only_in_rd    = rd_countries - trade_countries - emp_countries

# Print results
print("🔹 Countries only in Trade dataset:")
print(sorted(only_in_trade))

print("\n🔹 Countries only in Employment dataset:")
print(sorted(only_in_emp))

print("\n🔹 Countries only in R&D dataset:")
print(sorted(only_in_rd))

# Show all countries that don't match in all 3
all_countries = trade_countries.union(emp_countries).union(rd_countries)
in_all_three = trade_countries & emp_countries & rd_countries
mismatched_countries = all_countries - in_all_three

print("\n❗ Countries NOT present in all three datasets:")
print(sorted(mismatched_countries))


🔹 Countries only in Trade dataset:
['Mayotte', 'Other Asia, nes', 'Wallis and Futuna']

🔹 Countries only in Employment dataset:
['Anguilla', "CÃ´te d'Ivoire", 'Macao, China', 'Niue', 'Reunion', 'Taiwan, China', 'Tokelau']

🔹 Countries only in R&D dataset:
['Africa Eastern and Southern', 'Africa Western and Central', 'American Samoa', 'Arab World', 'British Virgin Islands', 'Caribbean small states', 'Central Europe and the Baltics', 'Chad', 'Channel Islands', 'Early-demographic dividend', 'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)', 'East Asia & Pacific (excluding high income)', 'Equatorial Guinea', 'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (excluding high income)', 'European Union', 'Fragile and conflict affected situations', 'Guam', 'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only', 'IDA total', 'Korea, Dem. Rep.', 'Late-demographic

The list of countries below represent not individual, but group of regions. They will not be included into final dataset

In [127]:
# List of non-country aggregates to drop
aggregates_to_drop = [
    'World', 'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)',
    'East Asia & Pacific (excluding high income)', 'Euro area', 'Europe & Central Asia',
    'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (excluding high income)',
    'European Union', 'Fragile and conflict affected situations',
    'Latin America & Caribbean', 'Latin America & Caribbean (excluding high income)',
    'Latin America & the Caribbean (IDA & IBRD countries)',
    'Sub-Saharan Africa', 'Sub-Saharan Africa (IDA & IBRD countries)',
    'Sub-Saharan Africa (excluding high income)', 'North America', 'OECD members',
    'Pacific island small states', 'South Asia', 'South Asia (IDA & IBRD)',
    'Middle East & North Africa', 'Middle East & North Africa (IDA & IBRD countries)',
    'Middle East & North Africa (excluding high income)', 'Other small states',
    'Other Asia, nes', 'Small states', 'Post-demographic dividend', 'Pre-demographic dividend',
    'IBRD only', 'IDA only', 'IDA blend', 'IDA & IBRD total', 'IDA total',
    'Least developed countries: UN classification', 'Low income', 'Middle income',
    'Upper middle income', 'Low & middle income', 'Late-demographic dividend',
    'High income', 'Not classified', 'Africa Eastern and Southern', 'Africa Western and Central',
    'Arab World', 'Caribbean small states', 'Central Europe and the Baltics',
    'Early-demographic dividend', 'Heavily indebted poor countries (HIPC)',
    'Lower middle income'
]


In [128]:
df_semicond_to_merge.shape  # to check the shape of the DataFrame before dropping aggregates

(9871, 16)

In [129]:
df_emp_to_merge.shape  # to check the shape of the DataFrame before dropping aggregates

(84426, 5)

In [130]:
df_RD_to_merge.shape  # to check the shape of the DataFrame before dropping aggregates

(6384, 3)

In [131]:
df_semicond_to_merge = df_semicond_to_merge[~df_semicond_to_merge['Country'].isin(aggregates_to_drop)]


In [132]:
df_emp_to_merge = df_emp_to_merge[~df_emp_to_merge['Country'].isin(aggregates_to_drop)]


In [133]:
df_RD_to_merge = df_RD_to_merge[~df_RD_to_merge['Country'].isin(aggregates_to_drop)]


In [134]:
df_semicond_to_merge.shape  # lost 71 rows after dropping aggregates

(9800, 16)

In [135]:
df_emp_to_merge.shape # lost 0 rows after dropping aggregates

(84426, 5)

In [136]:
df_RD_to_merge.shape  # lost 984 rows after dropping aggregates

(5208, 3)

In [137]:
# Compare countries pairwise
trade_countries = set(df_semicond_to_merge['Country'].unique())
emp_countries   = set(df_emp_to_merge['Country'].unique())
rd_countries    = set(df_RD_to_merge['Country'].unique())

only_in_trade = trade_countries - emp_countries - rd_countries
only_in_emp   = emp_countries - trade_countries - rd_countries
only_in_rd    = rd_countries - trade_countries - emp_countries

# Step 3: Print results
print("🔹 Countries only in Trade dataset:")
print(sorted(only_in_trade))

print("\n🔹 Countries only in Employment dataset:")
print(sorted(only_in_emp))

print("\n🔹 Countries only in R&D dataset:")
print(sorted(only_in_rd))

# Step 4 (optional): Show all countries that don't match in all 3
all_countries = trade_countries.union(emp_countries).union(rd_countries)
in_all_three = trade_countries & emp_countries & rd_countries
mismatched_countries = all_countries - in_all_three

print("\n❗ Countries NOT present in all three datasets:")
print(sorted(mismatched_countries))


🔹 Countries only in Trade dataset:
['Mayotte', 'Wallis and Futuna']

🔹 Countries only in Employment dataset:
['Anguilla', "CÃ´te d'Ivoire", 'Macao, China', 'Niue', 'Reunion', 'Taiwan, China', 'Tokelau']

🔹 Countries only in R&D dataset:
['American Samoa', 'British Virgin Islands', 'Chad', 'Channel Islands', 'Equatorial Guinea', 'Guam', 'Korea, Dem. Rep.', 'Liechtenstein', 'Marshall Islands', 'Monaco', 'Northern Mariana Islands', 'Puerto Rico', 'San Marino', 'Sint Maarten (Dutch part)', 'South Sudan', 'St. Lucia', 'St. Martin (French part)', 'Turkmenistan', 'Virgin Islands (U.S.)']

❗ Countries NOT present in all three datasets:
['American Samoa', 'Andorra', 'Anguilla', 'Argentina', 'Benin', 'Brazil', 'British Virgin Islands', 'Burundi', 'Central African Republic', 'Chad', 'Channel Islands', 'Colombia', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Cook Islands', "Cote d'Ivoire", "CÃ´te d'Ivoire", 'Djibouti', 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Faroe Islands', 'French Polynesia', 'Gabon'

* **Merging 3 datasets after countrywise corrections**

Machine learning models work best with complete, clean data. Inner join ensures model sees consistent information for all training examples.

Avoids introducing noise from partially missing countries

In [ ]:
# merging datasets by "inner" type join

df_final_ml = df_semicond_to_merge.merge(df_RD_to_merge, on=['Country', 'Year'], how='inner')
df_final_ml = df_final_ml.merge(df_emp_to_merge, on=['Country', 'Year'], how='inner')

In [139]:
# to get a concise summary of the final DataFrame

df_final_ml.info()                                    # Contains only countries that appear in all three datasets (Trade, Employment, R&D).

                                                        # Ensures a complete, high-quality feature set for modeling.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218333 entries, 0 to 218332
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Year                            218333 non-null  int64  
 1   Country                         218333 non-null  object 
 2   Trade_Flow_Type                 218333 non-null  object 
 3   Partner_Code                    218333 non-null  object 
 4   Commodity_Type                  218333 non-null  object 
 5   Alternative_Quantity_Value      205353 non-null  float64
 6   Net_Weight_Estimated            218333 non-null  float64
 7   Gross_Weight_Estimated          218333 non-null  float64
 8   Trade_Valuation_Source_Code     218333 non-null  int64  
 9   Legacy_Estimation_Method        218333 non-null  bool   
 10  Officially_Reported_by_Country  218333 non-null  bool   
 11  Trade_Value_Is_Estimated        218333 non-null  bool   
 12  Alt_Quantity_Is_

**df_ml_inner_join.csv**

In [ ]:
# I have commented this line to avoid saving the file again, as it is already saved 

# df_final_ml.to_csv("../Data/Processed/df_ml_inner_join.csv", index=False)  # Save the final DataFrame for ML to a CSV file

A left join preserves all available Employment_Change_Rate values, maximizing training data and retaining full 2000–2023 coverage where possible. 

It also reduces bias by keeping countries with incomplete trade or R&D data in the analysis.

Therefore, a 'left' join can be used for modeling, but it will still be checked to ensure it provides good model performance.

In [ ]:
df_left_merged = df_emp_to_merge.merge(df_RD_to_merge, on=['Country', 'Year'], how='left')    
df_left_merged = df_left_merged.merge(df_semicond_to_merge, on=['Country', 'Year'], how='left')

In [142]:
# Check result
df_left_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232782 entries, 0 to 232781
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Country                         232782 non-null  object 
 1   Gender                          232782 non-null  object 
 2   Occupation                      232782 non-null  object 
 3   Year                            232782 non-null  int64  
 4   Employment_Count                232782 non-null  float64
 5   RD_Expenditure_GDP_Percent      174697 non-null  float64
 6   Trade_Flow_Type                 219543 non-null  object 
 7   Partner_Code                    219543 non-null  object 
 8   Commodity_Type                  219543 non-null  object 
 9   Alternative_Quantity_Value      206563 non-null  float64
 10  Net_Weight_Estimated            219543 non-null  float64
 11  Gross_Weight_Estimated          219543 non-null  float64
 12  Trade_Valuation_

In [143]:
df_left_merged.isna().sum()  # Check for NaN values in the final DataFrame

Country                               0
Gender                                0
Occupation                            0
Year                                  0
Employment_Count                      0
RD_Expenditure_GDP_Percent        58085
Trade_Flow_Type                   13239
Partner_Code                      13239
Commodity_Type                    13239
Alternative_Quantity_Value        26219
Net_Weight_Estimated              13239
Gross_Weight_Estimated            13239
Trade_Valuation_Source_Code       13239
Legacy_Estimation_Method          13239
Officially_Reported_by_Country    13239
Trade_Value_Is_Estimated          13239
Alt_Quantity_Is_Estimated         13239
Net_Weight_Is_Estimated           13239
Gross_Weight_Is_Estimated         13239
Trade_Value_usd                   13239
dtype: int64

**df_ml_left_join.csv**

In [ ]:
#I have commented this line to avoid saving the file again, as it is already saved 

# df_left_merged.to_csv("../Data/Processed/df_ml_left_join.csv", index=False)  # Save the final DataFrame for ML to a CSV file

**To be continued in the next notebook**